In [4]:
import yfinance as yf
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [62]:
class Update_Stock():

    
    def __init__(self,market) :
        
        self.location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite'
        self.location_driver = r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe"
        self.Market = str(market)

#--------------------------------------- NASDAQ CRYPTO------------------------------------------------------------


    def Set_up_WebScraping_Info_NASDAQ_CRYPTO(self,share):
        
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
        driver.get(f"https://finance.yahoo.com/quote/{str(share)}/profile")
        time.sleep(2)

        try :
            driver.find_element(By.XPATH, '//*[@id="myLightboxContainer"]/section/button[2]').click()
            return str(driver.page_source)
        
        except NoSuchElementException :

            return ''

    def get_new_NASDAQ_CRYPTO_info(self,share):
        
        info = {}
        html = self.Set_up_WebScraping_Info_NASDAQ_CRYPTO(share)
        soup = BeautifulSoup(html, 'html')
        company = soup.find('h1', {'class': 'D(ib) Fz(18px)'})

        if str(company) == 'None' : return {}

        info['Symbol'],info['Sname'] = share,(company.text).split(f'({share})')[0][:-1]
        target =  soup.find('div', {'class': 'Pt(10px) smartphone_Pt(20px) Lh(1.7)'})
        info['MarketId'] = 3
        if str(target) != str(None):

            ind_sec = target.find_all('p',{'class':'D(ib) Va(t)'})
            chest = ind_sec[0].find_all('span',{'class':'Fw(600)'})[:-1]
            info['Sector'] = chest[0].text
            info['Industry'] = chest[1].text
            info['MarketId'] = 2

        return info

#--------------------------------------- SET ------------------------------------------------------------
# SET 
    def Set_up_WebScraping_info_SET(self,url):
        
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
        driver.get(url)
        
        return str(driver.page_source)

    def filter_info(self,text):
        company = ''
        fil = text.replace('\n','').split(' ')
        for word in fil : 
            if word != '' : 
                company += word + ' '

        return company[:-1]

    def get_SET_info(self,share):
        
        info,share = {},str(share)
        html = self.Set_up_WebScraping_info_SET(f"https://www.settrade.com/en/equities/quote/{share}/company-profile/profile")
        soup = BeautifulSoup(html, 'html')
        company_data = soup.find('div', {'class': 'company-cover bg-white border rounded box-shadow p-4 pt-0'})

        if str(company_data) == 'None' : return {}

        info['Symbol'],info['Sname'],info['MarketId'] = share+'.BK',filter_info(company_data.find('h3').text),1
        
        Industry_data = soup.find('div', {'class': 'col-12 col-md-6 order-3 order-md-3 mb-3'})
        info['Industry'] = self.filter_info(Industry_data.find('div', {'class': 'fs-14px text-middle-gray m-0'}).text)
        Sector_data = soup.find('div', {'class': 'col-12 col-md-6 order-4 order-md-4 mb-3'})
        info['Sector'] = self.filter_info(Sector_data.find('div', {'class': 'fs-14px text-middle-gray m-0'}).text)

        return info

#--------------------------------------- For all------------------------------------------------------------
    def add_share_info(self,data):
        
        conn = sqlite3.connect(self.location_db)
        
        cur = conn.cursor()
        cur.execute(f"""SELECT count(*) FROM Information WHERE Symbol = '{data['Symbol']}'""")
        records = cur.fetchone()
        
        if records[0] == 0 :

            cur.execute(f"""INSERT INTO Information (Symbol,Sname,MarketId) VALUES (?,?,?)""",(data['Symbol'],data['Sname'],data['MarketId']))
            conn.commit()
            cur.execute(f"Select SymbolId from Information where Symbol = '{data['Symbol']}'")
            records = cur.fetchone()
            
            data['SymbolId'] = records[0]
            conn.close()

            return data
        
        conn.close()
        return {}

    def get_sector(self,data):

        conn = sqlite3.connect(self.location_db)
        
        cur = conn.cursor()
        cur.execute(f"""SELECT count(*),SectorId FROM Sector WHERE Short_Sector = '{data['Sector']}' or Full_Sector = '{data['Sector']}' and MarketId = '{data['MarketId']}' """)
        records = cur.fetchone()
        data['SectorId'] = None
        conn.close()
        if records[0] == 1 :
            data['SectorId'] = records[1]

            return data
        
        return data

    def get_Industry(self,data):

        conn = sqlite3.connect(self.location_db)
        
        cur = conn.cursor()
        cur.execute(f"""SELECT count(*),IndustryId FROM Industry WHERE Short_Industry = '{data['Industry']}' or Full_Industry = '{data['Industry']}' and MarketId = '{data['MarketId']}' """)
        records = cur.fetchone()
        data['IndustryId'] = None
        conn.close()
        if records[0] == 1 :
            data['IndustryId'] = records[1]

            return data
        
        return data

    def add_Category_share(self,data):

        conn = sqlite3.connect(self.location_db)
        
        cur = conn.cursor()

        cur.execute(f"""INSERT INTO Category (SymbolId,SectorId,IndustryId) VALUES (?,?,?)""",(data['SymbolId'],data['SectorId'],data['IndustryId']))

        conn.commit()
        cur.execute(f"Select SymbolId from Information where Symbol = '{data['Symbol']}'")
        records = cur.fetchone()
            
        data['SymbolId'] = records[0]

        return data
    
    def add_new_symbol(self,share):

        if self.Market not in ['SET','NASDAQ','CRYPTO'] : return 'Available markets include: SET, NASDAQ, CRYPTO'
        if self.Market == 'SET' : data = self.get_SET_info(share)
        if self.Market == 'NASDAQ' : data = self.get_new_NASDAQ_CRYPTO_info(share)
        if self.Market == 'CRYPTO'  : data = self.get_new_NASDAQ_CRYPTO_info(share+'-USD')
        
        if data == {}: return 'Something wrong'

        data = self.add_share_info(data) 
        
        if data == {}: return f'{share} already in database'
        print(data)
        if self.Market != 'CRYPTO' :

            data = self.get_sector(data)
            data = self.get_Industry(data)
            print(data)
            data = self.add_Category_share(data)

        return data


In [57]:
#NASDAQ CRYPTO

def Set_up_WebScraping_Info_NASDAQ_CRYPTO(share):
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
    driver.get(f"https://finance.yahoo.com/quote/{str(share)}/profile")
    time.sleep(2)

    try :
        driver.find_element(By.XPATH, '//*[@id="myLightboxContainer"]/section/button[2]').click()
        return str(driver.page_source)
    
    except NoSuchElementException :

        return ''

def get_new_NASDAQ_CRYPTO_info(share):
    
    info = {}
    html = Set_up_WebScraping_Info_NASDAQ_CRYPTO(share)
    soup = BeautifulSoup(html, 'html')
    company = soup.find('h1', {'class': 'D(ib) Fz(18px)'})

    if str(company) == 'None' : return {}

    info['Symbol'],info['Sname'] = share,(company.text).split(f'({share})')[0][:-1]
    target =  soup.find('div', {'class': 'Pt(10px) smartphone_Pt(20px) Lh(1.7)'})
    info['MarketId'] = 3
    if str(target) != str(None):

        ind_sec = target.find_all('p',{'class':'D(ib) Va(t)'})
        chest = ind_sec[0].find_all('span',{'class':'Fw(600)'})[:-1]
        info['Sector'] = chest[0].text
        info['Industry'] = chest[1].text
        info['MarketId'] = 2

    return info


In [58]:
# SET 
def Set_up_WebScraping_info_SET(url):
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
    driver.get(url)
    
    return str(driver.page_source)

def filter_info(text):
    company = ''
    fil = text.replace('\n','').split(' ')
    for word in fil : 
        if word != '' : 
            company += word + ' '

    return company[:-1]

def get_SET_info(share):
    
    info,share = {},str(share)
    html = Set_up_WebScraping_info_SET(f"https://www.settrade.com/en/equities/quote/{share}/company-profile/profile")
    soup = BeautifulSoup(html, 'html')
    company_data = soup.find('div', {'class': 'company-cover bg-white border rounded box-shadow p-4 pt-0'})

    if str(company_data) == 'None' : return {}

    info['Symbol'],info['Sname'],info['MarketId'] = share+'.BK',filter_info(company_data.find('h3').text),1
    
    Industry_data = soup.find('div', {'class': 'col-12 col-md-6 order-3 order-md-3 mb-3'})
    info['Industry'] = filter_info(Industry_data.find('div', {'class': 'fs-14px text-middle-gray m-0'}).text)
    Sector_data = soup.find('div', {'class': 'col-12 col-md-6 order-4 order-md-4 mb-3'})
    info['Sector'] = filter_info(Sector_data.find('div', {'class': 'fs-14px text-middle-gray m-0'}).text)

    return info


In [59]:
# ฟังก์ชั่นเพิ่ม หุ้น

def add_share_info(data):
    
    conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
    
    cur = conn.cursor()
    cur.execute(f"""SELECT count(*) FROM Information WHERE Symbol = '{data['Symbol']}'""")
    records = cur.fetchone()
    
    if records[0] == 0 :

        cur.execute(f"""INSERT INTO Information (Symbol,Sname,MarketId) VALUES (?,?,?)""",(data['Symbol'],data['Sname'],data['MarketId']))
        conn.commit()
        cur.execute(f"Select SymbolId from Information where Symbol = '{data['Symbol']}'")
        records = cur.fetchone()
        
        data['SymbolId'] = records[0]
        conn.close()

        return data
    
    conn.close()
    return {}

def get_sector(data):

    conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
    
    cur = conn.cursor()
    cur.execute(f"""SELECT count(*),SectorId FROM Sector WHERE Short_Sector = '{data['Sector']}' or Full_Sector = '{data['Sector']}' and MarketId = '{data['MarketId']}' """)
    records = cur.fetchone()
    data['SectorId'] = None
    conn.close()
    if records[0] == 1 :
        data['SectorId'] = records[1]

        return data
    
    return data

def get_Industry(data):

    conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
    
    cur = conn.cursor()
    cur.execute(f"""SELECT count(*),IndustryId FROM Industry WHERE Short_Industry = '{data['Industry']}' or Full_Industry = '{data['Industry']}' and MarketId = '{data['MarketId']}' """)
    records = cur.fetchone()
    data['IndustryId'] = None
    conn.close()
    if records[0] == 1 :
        data['IndustryId'] = records[1]

        return data
    
    return data

def add_Category_share(data):

    conn = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
    
    cur = conn.cursor()

    cur.execute(f"""INSERT INTO Category (SymbolId,SectorId,IndustryId) VALUES (?,?,?)""",(data['SymbolId'],data['SectorId'],data['IndustryId']))

    conn.commit()
    cur.execute(f"Select SymbolId from Category where Symbol = '{data['Symbol']}'")
    records = cur.fetchone()
        
    data['SymbolId'] = records[0]

    return data


In [60]:
def add_new_symbol(market,share):

    if market not in ['SET','NASDAQ','CRYPTO'] : return 'Available markets include: SET, NASDAQ, CRYPTO'
    if market == 'SET' : data = get_SET_info(share)
    if market == 'NASDAQ' : data = get_new_NASDAQ_CRYPTO_info(share)
    if market == 'CRYPTO'  : data = get_new_NASDAQ_CRYPTO_info(share+'-USD')

    print(data)
    
    if data == {}: return 'Something wrong'

    data = add_share_info(data) 
    
    if data == {}: return f'{share} already in database'
    print(data)

    if market != 'CRYPTO' :

        data = get_sector(data)
        print(data)
        data = get_Industry(data)
        print(data)
        data = add_Category_share(data)
        print(data)
    return data

In [50]:
add_new_symbol('NASDAQ','V')

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_24796\1909569010.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_24796\1909569010.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)


{'Symbol': 'V', 'Sname': 'Visa Inc.', 'MarketId': 2, 'Sector': 'Financial Services', 'Industry': 'Credit Services'}


'V already in database'

In [51]:
add_Category_share({'Symbol': 'V', 'Sname': 'Visa Inc.', 'MarketId': 2, 'Sector': 'Financial Services', 'Industry': 'Credit Services', 'SymbolId': 5216, 'SectorId': None, 'IndustryId': None})

{'Symbol': 'V',
 'Sname': 'Visa Inc.',
 'MarketId': 2,
 'Sector': 'Financial Services',
 'Industry': 'Credit Services',
 'SymbolId': 5216,
 'SectorId': None,
 'IndustryId': None}

---------------------------HOW to USE---------------------------

In [61]:
Update_Stock('NASDAQ').add_new_symbol('MA')

C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_24796\31201649.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)
C:\Users\JourneyQ\AppData\Local\Temp\ipykernel_24796\31201649.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Schap\chromedriver.exe", chrome_options=options)


OperationalError: no such column: Symbol